# Importing Libraries

In [54]:
import sys
import os
import json
import numpy as np
from tqdm import tqdm
import torch
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils_data import  generateTargetDataBuySide, generateTargetDataSellSide, getTechnicalIndicators, normalize_dataframe_with_mean_std, normalize_new_row_with_mean_std
from utils_data import UpstoxStockDataFetcher

# Load data

In [55]:
fetcher = UpstoxStockDataFetcher()
stock_symbol = "SUZLON"
start_date = "2025-06-15"
end_date = "2025-06-20"
df = fetcher.get_stock_data(stock_symbol, start_date, end_date)
df 


,time,open,high,low,close,volume
0,2025-06-16 09:15:00+05:30,64.50,65.05,64.45,65.05,1860104
1,2025-06-16 09:16:00+05:30,65.07,65.30,64.86,64.97,1193739
2,2025-06-16 09:17:00+05:30,64.96,65.07,64.88,64.92,571128
3,2025-06-16 09:18:00+05:30,64.95,65.01,64.60,64.66,772514
4,2025-06-16 09:19:00+05:30,64.66,64.69,64.38,64.57,1089367
...,...,...,...,...,...,...
1870,2025-06-20 15:25:00+05:30,63.05,63.13,62.88,62.98,2930500
1871,2025-06-20 15:26:00+05:30,62.98,62.98,62.53,62.63,3086615
1872,2025-06-20 15:27:00+05:30,62.62,63.00,62.50,62.75,4457938
1873,2025-06-20 15:28:00+05:30,62.75,62.85,62.55,62.77,2105004


In [56]:
df = getTechnicalIndicators(df)
target_buy = generateTargetDataBuySide(df,1.005,0.99)
target_sell = generateTargetDataSellSide(df,0.995,1.01)


Processing Sell Side Data: 100%|██████████| 1848/1848 [00:40<00:00, 45.83it/s] 


In [57]:
count1 = len(target_buy[target_buy['action'].isin(['End of Day'])])
count2 = len(target_buy[target_buy['action'].isin(['Target Hit'])])
count3 = len(target_buy[target_buy['action'].isin(['Stop Loss Hit'])])

print(f"End of Day: {count1}")
print(f"Target Hit: {count2}")
print(f"Stop Loss Hit: {count3}")

End of Day: 369
Target Hit: 960
Stop Loss Hit: 519


In [59]:
count4 = len(target_sell[target_sell['action'].isin(['End of Day'])])
count5 = len(target_sell[target_sell['action'].isin(['Target Hit'])])
count6 = len(target_sell[target_sell['action'].isin(['Stop Loss Hit'])])

print(f"End of Day: {count4}")
print(f"Target Hit: {count5}")
print(f"Stop Loss Hit: {count6}")

End of Day: 431
Target Hit: 1053
Stop Loss Hit: 364


In [60]:
with open(r'C:\Users\srija\Assignment\Trading\Training_strategies\json_files\suzlon_2025_06_15_norm_params.json', 'r') as f:
    norm_param = json.load(f)
df_normalized = df.apply(lambda x:  normalize_new_row_with_mean_std(x,norm_param), axis=1)
df_normalized 

,time,open,high,low,close,volume,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
0,42,63.42,63.54,63.40,63.54,0.108930,-0.266533,-2.303707,-5.240184,0.109099,-1.205820,2.163937,-0.447285,4.331221,-1.369187,-3.489320
1,43,63.52,63.73,63.46,63.73,-0.166159,-0.321184,-1.520403,-4.660122,-0.021074,-1.164472,1.845678,-0.135825,4.350442,0.746441,-3.433212
2,44,63.73,63.74,63.53,63.58,0.425413,-0.378733,-1.851354,-4.361051,-0.151750,-1.130492,1.544933,-0.130121,4.257465,0.311857,-3.547103
3,45,63.56,63.59,63.35,63.45,1.293563,-0.439594,-2.115113,-4.257528,-0.272122,-1.109814,1.349145,-0.461946,4.226542,-0.232476,-3.745794
4,46,63.45,63.45,63.31,63.35,0.704772,-0.493831,-2.306072,-4.266380,-0.378340,-1.102781,1.184487,-0.698218,4.013121,-0.178408,-3.886972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,385,63.05,63.13,62.88,62.98,11.373028,-1.012600,-2.564606,-3.566758,-1.063774,-1.324996,1.597117,-0.908112,1.698986,-1.979521,2.563809
1844,386,62.98,62.98,62.53,62.63,12.018350,-1.083812,-3.348106,-3.978885,-1.029466,-1.450082,1.659515,-1.830413,2.238697,-3.781702,1.317547
1845,387,62.62,63.00,62.50,62.75,17.686894,-1.147157,-2.691109,-4.086819,-1.031855,-1.517346,1.727251,-1.501692,2.832211,-2.639072,3.117499
1846,388,62.75,62.85,62.55,62.77,7.960731,-1.197254,-2.585738,-4.095237,-1.049480,-1.567676,1.790150,-1.362039,3.013916,-1.931792,3.967423


In [61]:
def get_state(df, current_step):
    row = df.iloc[current_step]
    state = np.array([
        row['time'],
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        row['volume'],
        row['MA50'],
        row['RSI'],
        row['MACD'],
        row['BB_upper'],
        row['BB_lower'],
        row['ADX'],
        row['CCI'],
        row['ATR'],
        row['ROC'],
        row['OBV']
    ], dtype=np.float32)
    return state

# Loading Model

In [62]:
from Models.DQN import DQN,DQNAgent
from trading_environment import StockTradingEnv

policy_net = DQN(16, 3)
target_net = DQN(16, 3)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [63]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  #'cuda' if torch.cuda.is_available() else

policy_net.to(device)
target_net.to(device)

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

In [86]:
model_path = r"C:\Users\srija\Assignment\Trading\Training_strategies\trained_models\suzlon_2025_06_15\suzlon_2025_06_15_1461.pth"
policy_net.load_state_dict(torch.load(model_path))
policy_net.eval()

DQN(
  (fc1): Linear(in_features=16, out_features=1440, bias=True)
  (fc2): Linear(in_features=1440, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=32, bias=True)
  (fc8): Linear(in_features=32, out_features=3, bias=True)
)

# Evaluate Model

In [87]:
import time
correct = 0
wrong = 0
blunder =0
latency_list = []
action_list = []
blunder_action = []
for i in range(0,len(df_normalized)):
    t1 = time.time()
    state = get_state(df_normalized,i)
    with torch.no_grad():
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        q_values = policy_net(state)
        action=q_values.argmax().item()
    action_list.append(action)
    ideal_action = 0
    if target_buy.iloc[i]['action'] == "Target Hit":
        ideal_action = 1
    elif target_sell.iloc[i]['action'] == "Target Hit":
        ideal_action = 2
    if action==ideal_action:
        correct += 1
    elif action == 1 and ideal_action ==2:
        blunder_action.append(action)
        blunder += 1
    elif action == 2 and ideal_action ==1:
        blunder_action.append(action)
        blunder += 1    
    else:
        wrong += 1
    t2 = time.time()
    latency = t2 - t1
    latency_list.append(latency)

print(f"Correct: {correct}, Wrong: {wrong}, Blunder: {blunder}")  
# print(f"Average Latency: {np.mean(latency_list)} seconds")  

print(f"Action Distribution: {np.bincount(action_list)}")  # Count occurrences
print(f"Blunder Action Distribution: {np.bincount(blunder_action)}")  # Count occurrences

Correct: 627, Wrong: 929, Blunder: 292
Action Distribution: [1036  436  376]
Blunder Action Distribution: [  0 136 156]


In [ ]:
# Correct: 683, Wrong: 183, Blunder: 232
# Action Distribution: [189 506 403]
# Blunder Action Distribution: [  0  86 146]